In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsmodels.tsa.seasonal import MSTL
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Exponential Smoothing

In [ ]:
LOCATION = "Nelson St"

In [ ]:
cycle_counts = pd.read_csv("cycle_counts.csv", parse_dates=["time"])
cycle_counts = cycle_counts[cycle_counts["location"] == LOCATION]

In [ ]:
fig, ax = plt.subplots()
ax.plot(cycle_counts["time"], cycle_counts["count"])
ax.set(title=LOCATION, ylabel="Count")
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
fig.tight_layout();

## STL Decomposition

In [ ]:
cycle_counts["time"] = pd.to_datetime(cycle_counts["time"])
cycle_counts = cycle_counts.set_index("time").drop(columns=["location"])
cycle_counts = cycle_counts.resample("D").sum()
cycle_counts = cycle_counts.fillna(0)

In [ ]:
stl = MSTL(cycle_counts["count"], periods=7)
result = stl.fit()
result.plot();

## Exponential Smoothing

In [ ]:
model = ExponentialSmoothing(
    cycle_counts["count"],
    trend=None,
    damped_trend=False,
    seasonal="add",
    seasonal_periods=7
)
model = model.fit()
y_hat = model.predict(start=cycle_counts.index[0], end=cycle_counts.index[-1])


In [ ]:
fig, ax = plt.subplots()
ax.plot(cycle_counts["count"], label="Observed")
ax.plot(y_hat, label="Forecast")
ax.set(title=LOCATION, ylabel="Count")
for tick in ax.get_xticklabels():
    tick.set_rotation(45)
ax.legend()
fig.tight_layout();

## Cross-validation

In [ ]:
class TimeSeriesSplitter:
    def __init__(self, n_splits: int, window: int, gap: int):
        self.n_splits = n_splits
        self.window = window
        self.gap = gap

    def split(self, y: pd.Series):
        n = len(y)
        start_idx = n - self.window - self.gap * (self.n_splits - 1)
        for _ in range(self.n_splits):
            end_idx = start_idx + self.window
            yield start_idx, end_idx
            start_idx = start_idx + self.gap

In [ ]:
actuals = []
forecasts = []

## Residuals